In [26]:
# ============================================================================
# CELDA 1: IMPORTAR LIBRERÍAS
# ============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import os

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [27]:
# ============================================================================
# CELDA 2: CONFIGURAR RUTAS
# ============================================================================

BASE_PATH = r'C:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final'
INPUT_FILE = os.path.join(BASE_PATH, 'data', 'processed', 'crypto_stocks_with_indices_complete.csv')
OUTPUT_DIR = os.path.join(BASE_PATH, 'data', 'powerbi')

# Crear carpeta de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Ruta base: {BASE_PATH}")
print(f"📂 Carpeta salida: {OUTPUT_DIR}")
print(f"✅ Configuración completa")
# ============================================================================
# CONFIGURACIÓN REGIONAL PARA POWER BI
# ============================================================================

# Forzar formato inglés (punto como decimal)
import locale
try:
    locale.setlocale(locale.LC_NUMERIC, 'C')
except:
    pass

print("✅ Configuración regional: Formato inglés (punto decimal)")

📁 Ruta base: C:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final
📂 Carpeta salida: C:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final\data\powerbi
✅ Configuración completa
✅ Configuración regional: Formato inglés (punto decimal)


In [28]:
# ============================================================================
# CELDA 3: CARGAR DATOS PRINCIPALES
# ============================================================================
print("="*70)
print("🚀 CARGANDO DATASET PRINCIPAL")
print("="*70)

df = pd.read_csv(INPUT_FILE)
df['date'] = pd.to_datetime(df['date'])

print(f"\n✅ Dataset cargado exitosamente")
print(f"   📊 Filas: {len(df):,}")
print(f"   📋 Columnas: {len(df.columns)}")
print(f"   📅 Período: {df['date'].min().date()} a {df['date'].max().date()}")
print(f"\n📋 Columnas disponibles:")
for col in df.columns:
    print(f"   • {col}")

# Vista previa
df.head(3)

🚀 CARGANDO DATASET PRINCIPAL

✅ Dataset cargado exitosamente
   📊 Filas: 50,109
   📋 Columnas: 18
   📅 Período: 2012-01-03 a 2024-12-30

📋 Columnas disponibles:
   • date
   • close
   • high
   • low
   • open
   • volume
   • asset_name
   • asset_class
   • symbol
   • nasdaq_10_index
   • nasdaq_10_ma8
   • nasdaq_10_ma21
   • crypto_7_index
   • crypto_7_ma8
   • crypto_7_ma21
   • nasdaq_10_returns
   • crypto_7_returns
   • crypto_nasdaq_ratio


,date,close,high,low,open,volume,asset_name,asset_class,symbol,nasdaq_10_index,nasdaq_10_ma8,nasdaq_10_ma21,crypto_7_index,crypto_7_ma8,crypto_7_ma21,nasdaq_10_returns,crypto_7_returns,crypto_nasdaq_ratio
0,2014-09-17,457.334015,468.174011,452.421997,465.864014,21056800,BITCOIN,crypto,BTC-USD,196.598732,198.497255,198.559788,NaN,NaN,NaN,0.001141,NaN,NaN
1,2014-09-18,424.440002,456.859985,413.104004,456.859985,34483200,BITCOIN,crypto,BTC-USD,198.101132,198.071618,198.586641,NaN,NaN,NaN,0.007642,NaN,NaN
2,2014-09-19,394.795990,427.834991,384.532013,424.102997,37919700,BITCOIN,crypto,BTC-USD,197.689960,197.887530,198.594963,NaN,NaN,NaN,-0.002076,NaN,NaN


In [29]:
# ============================================================================
# CELDA 4: TABLA 1 - DATOS DIARIOS
# ============================================================================
print("\n" + "="*70)
print("📊 TABLA 1: DATOS DIARIOS")
print("="*70)

daily_data = df[[
    'date', 'asset_name', 'asset_class', 'close', 'volume',
    'crypto_7_index', 'nasdaq_10_index'
]].copy()

# Agregar columnas de tiempo para filtros
daily_data['year'] = daily_data['date'].dt.year
daily_data['month'] = daily_data['date'].dt.month
daily_data['year_month'] = daily_data['date'].dt.to_period('M').astype(str)

# Calcular retornos diarios
daily_data['daily_return'] = daily_data.groupby('asset_name')['close'].pct_change()

# Guardar
output_file = os.path.join(OUTPUT_DIR, '01_daily_data.csv')
daily_data.to_csv(output_file, index=False, encoding='utf-8-sig', decimal='.')

print(f"✅ Guardado: 01_daily_data.csv")
print(f"   📊 Filas: {len(daily_data):,}")
print(f"   📋 Columnas: {len(daily_data.columns)}")

# Vista previa
daily_data.head(3)



📊 TABLA 1: DATOS DIARIOS
✅ Guardado: 01_daily_data.csv
   📊 Filas: 50,109
   📋 Columnas: 11


,date,asset_name,asset_class,close,volume,crypto_7_index,nasdaq_10_index,year,month,year_month,daily_return
0,2014-09-17,BITCOIN,crypto,457.334015,21056800,NaN,196.598732,2014,9,2014-09,NaN
1,2014-09-18,BITCOIN,crypto,424.440002,34483200,NaN,198.101132,2014,9,2014-09,-0.071926
2,2014-09-19,BITCOIN,crypto,394.795990,37919700,NaN,197.689960,2014,9,2014-09,-0.069843


In [30]:
# ============================================================================
# CELDA 5: TABLA 2 - RESUMEN POR ACTIVO
# ============================================================================
print("\n" + "="*70)
print("📊 TABLA 2: RESUMEN POR ACTIVO")
print("="*70)

# Filtrar período común (2020-08-20 en adelante)
df_common = df[df['date'] >= '2020-08-20'].copy()

asset_summary = []

for asset in df_common['asset_name'].unique():
    asset_data = df_common[df_common['asset_name'] == asset].copy()
    
    # Calcular métricas
    precio_inicial = asset_data['close'].iloc[0]
    precio_final = asset_data['close'].iloc[-1]
    retorno_total = ((precio_final / precio_inicial) - 1) * 100
    
    returns = asset_data['close'].pct_change().dropna()
    volatilidad = returns.std() * np.sqrt(252) * 100  # Anualizada
    sharpe = (returns.mean() * 252) / (returns.std() * np.sqrt(252)) if returns.std() > 0 else 0
    
    max_drawdown = ((asset_data['close'] / asset_data['close'].cummax()) - 1).min() * 100
    
    asset_summary.append({
        'asset_name': asset,
        'asset_class': asset_data['asset_class'].iloc[0],
        'precio_inicial': round(precio_inicial, 2),
        'precio_final': round(precio_final, 2),
        'retorno_total_pct': round(retorno_total, 2),
        'volatilidad_anual_pct': round(volatilidad, 2),
        'sharpe_ratio': round(sharpe, 2),
        'max_drawdown_pct': round(max_drawdown, 2),
        'dias_operados': len(asset_data)
    })

asset_summary_df = pd.DataFrame(asset_summary)

# Guardar
output_file = os.path.join(OUTPUT_DIR, '02_asset_summary.csv')
asset_summary_df.to_csv(output_file, index=False, encoding='utf-8-sig', decimal='.')

print(f"✅ Guardado: 02_asset_summary.csv")
print(f"   📊 Activos: {len(asset_summary_df)}")

# Mostrar tabla ordenada por retorno
asset_summary_df.sort_values('retorno_total_pct', ascending=False)


📊 TABLA 2: RESUMEN POR ACTIVO
✅ Guardado: 02_asset_summary.csv
   📊 Activos: 17


,asset_name,asset_class,precio_inicial,precio_final,retorno_total_pct,volatilidad_anual_pct,sharpe_ratio,max_drawdown_pct,dias_operados
5,DOGECOIN,crypto,0.00,0.31,8963.72,180.93,0.89,-92.26,1594
4,SOLANA,crypto,3.21,191.03,5854.24,104.36,1.14,-96.27,1594
2,BINANCECOIN,crypto,23.09,704.15,2949.23,74.93,1.08,-70.84,1594
12,NVDA,stock,12.10,137.46,1035.99,52.20,1.33,-66.34,1097
1,ETHEREUM,crypto,416.44,3356.39,705.97,66.38,0.83,-79.35,1594
0,BITCOIN,crypto,11878.37,92643.21,679.93,51.13,0.89,-76.63,1594
3,CARDANO,crypto,0.13,0.86,543.52,80.85,0.76,-91.85,1594
8,TSLA,stock,133.46,417.41,212.77,61.80,0.73,-73.63,1097
11,GOOGL,stock,78.27,190.62,143.53,30.73,0.82,-44.32,1097
6,POLKADOT,crypto,2.90,6.70,130.91,89.71,0.58,-93.23,1594


In [31]:
# ============================================================================
# CELDA 6: TABLA 3 - ÍNDICES SINTÉTICOS
# ============================================================================
print("\n" + "="*70)
print("📊 TABLA 3: ÍNDICES SINTÉTICOS")
print("="*70)

indices_data = df[df['date'] >= '2020-08-20'][[
    'date', 'crypto_7_index', 'nasdaq_10_index',
    'crypto_7_ma8', 'crypto_7_ma21',
    'nasdaq_10_ma8', 'nasdaq_10_ma21'
]].copy()

# Eliminar duplicados
indices_data = indices_data.drop_duplicates('date').reset_index(drop=True)

# Calcular retornos
indices_data['crypto_7_return'] = indices_data['crypto_7_index'].pct_change()
indices_data['nasdaq_10_return'] = indices_data['nasdaq_10_index'].pct_change()

# Agregar columnas de tiempo
indices_data['year'] = indices_data['date'].dt.year
indices_data['month'] = indices_data['date'].dt.month
indices_data['year_month'] = indices_data['date'].dt.to_period('M').astype(str)

# Guardar
output_file = os.path.join(OUTPUT_DIR, '03_indices_data.csv')
indices_data.to_csv(output_file, index=False, encoding='utf-8-sig', decimal='.')

print(f"✅ Guardado: 03_indices_data.csv")
print(f"   📊 Días: {len(indices_data):,}")

# Vista previa
indices_data.head(3)



📊 TABLA 3: ÍNDICES SINTÉTICOS
✅ Guardado: 03_indices_data.csv
   📊 Días: 1,594


,date,crypto_7_index,nasdaq_10_index,crypto_7_ma8,crypto_7_ma21,nasdaq_10_ma8,nasdaq_10_ma21,crypto_7_return,nasdaq_10_return,year,month,year_month
0,2020-08-20,100.000000,1418.484020,103.991617,105.183122,1369.488419,1316.294833,NaN,NaN,2020,8,2020-08
1,2020-08-21,95.492579,1437.951793,103.991617,105.183122,1387.652710,1326.482815,-0.045074,0.013724,2020,8,2020-08
2,2020-08-22,104.260916,1437.951793,103.991617,105.183122,1387.652710,1326.482815,0.091822,0.000000,2020,8,2020-08


In [48]:
# ============================================================================
# CELDA 7: TABLA 4 - ESTRATEGIAS (MÉTODO ORIGINAL - EXACTO)
# ============================================================================
print("\n" + "="*70)
print("📊 Creando Tabla 4: Estrategias de Inversión")
print("="*70)

# Filtrar al período común (desde 2020-08-20)
fecha_inicio_comun = '2020-08-20'
indices_comun = df[df['date'] >= fecha_inicio_comun].copy()

# Obtener datos únicos por fecha
indices = indices_comun.drop_duplicates('date')[['date', 'crypto_7_index', 'nasdaq_10_index']].sort_values('date').copy()
indices = indices.set_index('date')

# Calcular retornos diarios
indices['crypto_7_returns'] = indices['crypto_7_index'].pct_change()
indices['nasdaq_10_returns'] = indices['nasdaq_10_index'].pct_change()
indices = indices.dropna()

print(f"✅ Datos preparados: {len(indices)} días de trading")

# Definir las 7 estrategias
estrategias = {
    '100% CRYPTO': {'nasdaq': 0.00, 'crypto': 1.00},
    '70% CRYPTO - 30% NASDAQ': {'nasdaq': 0.30, 'crypto': 0.70},
    '50% CRYPTO - 50% NASDAQ': {'nasdaq': 0.50, 'crypto': 0.50},
    '30% CRYPTO - 70% NASDAQ': {'nasdaq': 0.70, 'crypto': 0.30},
    '100% NASDAQ': {'nasdaq': 1.00, 'crypto': 0.00},
    
    'Risk Parity (40/60)': {'nasdaq': 0.60, 'crypto': 0.40}
}

resultados = []
capital_inicial = 10000

for nombre, weights in estrategias.items():
    # Calcular retornos de cartera (MÉTODO ORIGINAL)
    portfolio_returns = (
        indices['nasdaq_10_returns'] * weights['nasdaq'] + 
        indices['crypto_7_returns'] * weights['crypto']
    )
    
    # Valor de cartera
    portfolio_value = capital_inicial * (1 + portfolio_returns).cumprod()
    final_value = portfolio_value.iloc[-1]
    total_return = (final_value / capital_inicial - 1) * 100
    
    # Volatilidad anual
    volatility = portfolio_returns.std() * np.sqrt(252) * 100
    
    # Sharpe Ratio (MÉTODO ORIGINAL - con tasa libre de riesgo 2%)
    risk_free = 0.02 / 252
    sharpe = ((portfolio_returns.mean() - risk_free) / portfolio_returns.std()) * np.sqrt(252)
    
    # Sortino Ratio
    negative_returns = portfolio_returns[portfolio_returns < 0]
    downside_std = negative_returns.std() * np.sqrt(252)
    sortino = ((portfolio_returns.mean() - risk_free) / negative_returns.std()) * np.sqrt(252) if len(negative_returns) > 0 else 0
    
    # Max Drawdown (MÉTODO ORIGINAL)
    cumulative = portfolio_value
    running_max = cumulative.cummax()
    drawdown = (cumulative - running_max) / running_max
    max_dd = drawdown.min() * 100
    
    # Calmar Ratio
    calmar = (total_return / abs(max_dd)) if max_dd != 0 else 0
    
    resultados.append({
        'estrategia': nombre,
        'peso_crypto_pct': weights['crypto'] * 100,
        'peso_nasdaq_pct': weights['nasdaq'] * 100,
        'capital_inicial': capital_inicial,
        'valor_final': final_value,
        'retorno_total_pct': total_return,
        'volatilidad_anual_pct': volatility,
        'sharpe_ratio': sharpe,
        'sortino_ratio': sortino,
        'max_drawdown_pct': max_dd,
        'calmar_ratio': calmar,
        'fecha_inicio': indices.index[0],
        'fecha_fin': indices.index[-1],
        'dias_trading': len(indices)
    })

# Crear DataFrame
estrategias_df = pd.DataFrame(resultados)

# Ordenar por Sharpe Ratio descendente
estrategias_df = estrategias_df.sort_values('sharpe_ratio', ascending=False).reset_index(drop=True)

# Guardar
output_file = os.path.join(OUTPUT_DIR, '04_estrategias.csv')
estrategias_df.to_csv(output_file, index=False, encoding='utf-8-sig', decimal='.')

# Imprimir resultados
print(f"✅ Guardado: 04_estrategias.csv")
print(f"   📊 Estrategias: {len(estrategias_df)}")

fecha_inicio = str(estrategias_df['fecha_inicio'].iloc[0]).split()[0]
fecha_fin = str(estrategias_df['fecha_fin'].iloc[0]).split()[0]
print(f"   📅 Período: {fecha_inicio} a {fecha_fin}")
print(f"   💰 Capital inicial: ${capital_inicial:,}")
print(f"\n🏆 RANKING POR SHARPE RATIO:")
for idx, row in estrategias_df.head(7).iterrows():
    nombre = row['estrategia']
    sharpe = row['sharpe_ratio']
    valor = row['valor_final']
    retorno = row['retorno_total_pct']
    print(f"   {idx+1}. {nombre:<30} | Sharpe: {sharpe:.2f} | Valor Final: ${valor:>12,.0f} | Retorno: {retorno:>8.1f}%")




📊 Creando Tabla 4: Estrategias de Inversión
✅ Datos preparados: 1593 días de trading
✅ Guardado: 04_estrategias.csv
   📊 Estrategias: 6
   📅 Período: 2020-08-21 a 2024-12-30
   💰 Capital inicial: $10,000

🏆 RANKING POR SHARPE RATIO:
   1. 70% CRYPTO - 30% NASDAQ        | Sharpe: 1.20 | Valor Final: $     235,319 | Retorno:   2253.2%
   2. 50% CRYPTO - 50% NASDAQ        | Sharpe: 1.19 | Valor Final: $     139,956 | Retorno:   1299.6%
   3. 100% CRYPTO                    | Sharpe: 1.18 | Valor Final: $     424,062 | Retorno:   4140.6%
   4. Risk Parity (40/60)            | Sharpe: 1.17 | Valor Final: $     103,901 | Retorno:    939.0%
   5. 30% CRYPTO - 70% NASDAQ        | Sharpe: 1.12 | Valor Final: $      75,193 | Retorno:    651.9%
   6. 100% NASDAQ                    | Sharpe: 0.61 | Valor Final: $      24,421 | Retorno:    144.2%


In [49]:
# ============================================================================
# CELDA 8: TABLA 5 - MATRIZ DE CORRELACIONES
# ============================================================================
print("\n" + "="*70)
print("📊 Creando Tabla 5: Matriz de Correlaciones")
print("="*70)

# Crear tabla de correlaciones entre activos
activos_pivot = df.pivot_table(
    index='date',
    columns='asset_name',
    values='close'
)

# Calcular retornos diarios
activos_returns = activos_pivot.pct_change().dropna()

# Calcular matriz de correlación
correlacion_matrix = activos_returns.corr()

# Convertir a formato largo para Power BI
correlaciones_list = []

for activo1 in correlacion_matrix.index:
    for activo2 in correlacion_matrix.columns:
        correlaciones_list.append({
            'activo_1': activo1,
            'activo_2': activo2,
            'correlacion': correlacion_matrix.loc[activo1, activo2]
        })

correlaciones_df = pd.DataFrame(correlaciones_list)

# Guardar
output_file = os.path.join(OUTPUT_DIR, '05_correlaciones.csv')
correlaciones_df.to_csv(output_file, index=False, encoding='utf-8-sig', decimal='.')

print(f"✅ Guardado: 05_correlaciones.csv")
print(f"   📊 Pares de correlación: {len(correlaciones_df)}")
print(f"   📋 Activos analizados: {len(correlacion_matrix)}")

# Mostrar algunas correlaciones interesantes
print(f"\n🔍 CORRELACIONES DESTACADAS:")

# Correlación CRYPTO 7 vs NASDAQ 10 (si existen en los datos)
if 'BITCOIN' in correlacion_matrix.index and 'AAPL' in correlacion_matrix.index:
    corr_btc_aapl = correlacion_matrix.loc['BITCOIN', 'AAPL']
    print(f"   BITCOIN vs AAPL: {corr_btc_aapl:.3f}")

if 'ETHEREUM' in correlacion_matrix.index and 'NVDA' in correlacion_matrix.index:
    corr_eth_nvda = correlacion_matrix.loc['ETHEREUM', 'NVDA']
    print(f"   ETHEREUM vs NVDA: {corr_eth_nvda:.3f}")

# Activo con mayor correlación promedio con otros
avg_correlations = correlacion_matrix.mean().sort_values(ascending=False)
print(f"\n📊 Activo con mayor correlación promedio:")
print(f"   {avg_correlations.index[0]}: {avg_correlations.iloc[0]:.3f}")

print(f"\n✅ Tabla 5 completada")


📊 Creando Tabla 5: Matriz de Correlaciones
✅ Guardado: 05_correlaciones.csv
   📊 Pares de correlación: 289
   📋 Activos analizados: 17

🔍 CORRELACIONES DESTACADAS:
   BITCOIN vs AAPL: 0.242
   ETHEREUM vs NVDA: 0.271

📊 Activo con mayor correlación promedio:
   MSFT: 0.462

✅ Tabla 5 completada


C:\Users\USER\AppData\Local\Temp\ipykernel_21716\3929536386.py:16: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  activos_returns = activos_pivot.pct_change().dropna()


In [36]:
# ============================================================================
# CELDA 9: TABLA 6 - EVENTOS DE CRISIS
# ============================================================================
print("\n" + "="*70)
print("📊 TABLA 6: EVENTOS HISTÓRICOS")
print("="*70)

eventos = [
    {'fecha': '2020-03-12', 'evento': 'COVID-19 Crash', 'tipo': 'Crisis'},
    {'fecha': '2021-05-19', 'evento': 'China Ban Crypto', 'tipo': 'Regulación'},
    {'fecha': '2022-05-09', 'evento': 'Terra/Luna Collapse', 'tipo': 'Crisis Crypto'},
    {'fecha': '2022-11-11', 'evento': 'FTX Bankruptcy', 'tipo': 'Crisis Crypto'},
    {'fecha': '2023-03-10', 'evento': 'SVB Bank Collapse', 'tipo': 'Crisis Financiera'},
    {'fecha': '2024-01-10', 'evento': 'Bitcoin ETF Approval', 'tipo': 'Hito Positivo'}
]

eventos_df = pd.DataFrame(eventos)
eventos_df['fecha'] = pd.to_datetime(eventos_df['fecha'])

# Guardar
output_file = os.path.join(OUTPUT_DIR, '06_eventos.csv')
eventos_df.to_csv(output_file, index=False, encoding='utf-8-sig', decimal='.')

print(f"✅ Guardado: 06_eventos.csv")
print(f"   📊 Eventos: {len(eventos_df)}")

# Mostrar tabla
eventos_df



📊 TABLA 6: EVENTOS HISTÓRICOS
✅ Guardado: 06_eventos.csv
   📊 Eventos: 6


,fecha,evento,tipo
0,2020-03-12,COVID-19 Crash,Crisis
1,2021-05-19,China Ban Crypto,Regulación
2,2022-05-09,Terra/Luna Collapse,Crisis Crypto
3,2022-11-11,FTX Bankruptcy,Crisis Crypto
4,2023-03-10,SVB Bank Collapse,Crisis Financiera
5,2024-01-10,Bitcoin ETF Approval,Hito Positivo


In [37]:
# ============================================================================
# CELDA 10: TABLA 7 - KPIs PRINCIPALES
# ============================================================================
print("\n" + "="*70)
print("📊 TABLA 7: KPIs PRINCIPALES")
print("="*70)

kpis = [
    {
        'metrica': 'CRYPTO 7 Return',
        'valor': 4140.6,
        'unidad': '%',
        'periodo': '2020-2024',
        'color': '#e74c3c'
    },
    {
        'metrica': 'NASDAQ 10 Return',
        'valor': 144.2,
        'unidad': '%',
        'periodo': '2020-2024',
        'color': '#3498db'
    },
    {
        'metrica': 'Mejor Estrategia',
        'valor': 50,
        'unidad': '% NASDAQ',
        'periodo': '50/50 Mix',
        'color': '#27ae60'
    },
    {
        'metrica': 'Sharpe Óptimo',
        'valor': 1.19,
        'unidad': 'ratio',
        'periodo': '50/50 Mix',
        'color': '#f39c12'
    },
    {
        'metrica': 'Correlación BTC-NASDAQ',
        'valor': 0.35,
        'unidad': 'coef',
        'periodo': '2020-2024',
        'color': '#9b59b6'
    },
    {
        'metrica': 'Bitcoin Lead Time',
        'valor': 3,
        'unidad': 'días',
        'periodo': 'Adelantado',
        'color': '#e67e22'
    }
]

kpis_df = pd.DataFrame(kpis)

# Guardar
output_file = os.path.join(OUTPUT_DIR, '07_kpis.csv')
kpis_df.to_csv(output_file, index=False, encoding='utf-8-sig', decimal='.')

print(f"✅ Guardado: 07_kpis.csv")
print(f"   📊 KPIs: {len(kpis_df)}")

# Mostrar tabla
kpis_df



📊 TABLA 7: KPIs PRINCIPALES
✅ Guardado: 07_kpis.csv
   📊 KPIs: 6


,metrica,valor,unidad,periodo,color
0,CRYPTO 7 Return,4140.60,%,2020-2024,#e74c3c
1,NASDAQ 10 Return,144.20,%,2020-2024,#3498db
2,Mejor Estrategia,50.00,% NASDAQ,50/50 Mix,#27ae60
3,Sharpe Óptimo,1.19,ratio,50/50 Mix,#f39c12
4,Correlación BTC-NASDAQ,0.35,coef,2020-2024,#9b59b6
5,Bitcoin Lead Time,3.00,días,Adelantado,#e67e22


In [38]:
# ============================================================================
# CELDA 11: RESUMEN FINAL Y VERIFICACIÓN
# ============================================================================
print("\n" + "="*70)
print("✅ PREPARACIÓN COMPLETADA")
print("="*70)

print(f"\n📁 Archivos generados en: {OUTPUT_DIR}\n")

# Verificar archivos creados
archivos_esperados = [
    '01_daily_data.csv',
    '02_asset_summary.csv',
    '03_indices_data.csv',
    '04_estrategias.csv',
    '05_correlaciones.csv',
    '06_eventos.csv',
    '07_kpis.csv'
]

print("📊 Verificación de archivos:\n")
for archivo in archivos_esperados:
    ruta = os.path.join(OUTPUT_DIR, archivo)
    existe = os.path.exists(ruta)
    tamaño = os.path.getsize(ruta) / 1024 if existe else 0  # KB
    
    if existe:
        print(f"   ✅ {archivo:<30} ({tamaño:.1f} KB)")
    else:
        print(f"   ❌ {archivo:<30} (NO ENCONTRADO)")




✅ PREPARACIÓN COMPLETADA

📁 Archivos generados en: C:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final\data\powerbi

📊 Verificación de archivos:

   ✅ 01_daily_data.csv              (5656.8 KB)
   ✅ 02_asset_summary.csv           (1.1 KB)
   ✅ 03_indices_data.csv            (272.9 KB)
   ✅ 04_estrategias.csv             (0.6 KB)
   ✅ 05_correlaciones.csv           (5.7 KB)
   ✅ 06_eventos.csv                 (0.3 KB)
   ✅ 07_kpis.csv                    (0.3 KB)


In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

# ============================================================================
# CREAR CARPETA ../data/powerbi/ (DESDE powerbi/)
# ============================================================================
ruta_powerbi = '../data/powerbi'
os.makedirs(ruta_powerbi, exist_ok=True)

print("\n" + "="*80)
print("📊 GENERANDO TABLAS CSV PARA POWER BI - CICLOS TEMPORALES")
print("="*80)
print(f"📁 Carpeta destino: {os.path.abspath(ruta_powerbi)}\n")

# ============================================================================
# TABLA 1: CICLOS BITCOIN - HALVINGS HISTÓRICOS
# ============================================================================
print("📋 Tabla 1: Ciclos Bitcoin - Halvings")

df_halvings = pd.DataFrame({
    'Halving_Numero': [1, 2, 3, 4],
    'Halving_Nombre': ['Halving #1', 'Halving #2', 'Halving #3', 'Halving #4 (Proyección)'],
    'Fecha_Halving': ['2012-11-28', '2016-07-09', '2020-05-11', '2024-04-20'],
    'Precio_Halving': [12, 650, 8787, 64000],
    'ATH_Precio': [1163, 19700, 69000, 160000],
    'ATH_Fecha': ['2013-11-30', '2017-12-17', '2021-11-10', '2025-10-15'],
    'Dias_HV_ATH': [368, 527, 549, 543],
    'Ganancia_Pct': [9319, 2931, 685, 150],
    'Fondo_Precio': [152, 3122, 15476, 35000],
    'Fondo_Fecha': ['2015-01-14', '2018-12-15', '2022-11-21', '2026-11-15'],
    'Drawdown_Pct': [-87, -84, -78, -78],
    'Tipo': ['Histórico', 'Histórico', 'Histórico', 'Proyección']
})

archivo = os.path.join(ruta_powerbi, 'ciclos_bitcoin_halvings.csv')
df_halvings.to_csv(archivo, index=False, encoding='utf-8-sig')
print(f"   ✅ ciclos_bitcoin_halvings.csv ({len(df_halvings)} filas)")

# ============================================================================
# TABLA 2: TIMELINE 2024-2027 - PROYECCIÓN DIARIA
# ============================================================================
print("📋 Tabla 2: Timeline Proyección 2024-2027")

# Generar fechas diarias
fecha_inicio = pd.to_datetime('2024-01-01')
fecha_fin = pd.to_datetime('2027-12-31')
fechas = pd.date_range(fecha_inicio, fecha_fin, freq='D')

# Proyección Bitcoin
btc_puntos = {
    '2024-01-01': 44000,
    '2024-04-20': 64000,
    '2024-12-31': 85000,
    '2025-06-30': 120000,
    '2025-10-15': 160000,
    '2025-12-31': 140000,
    '2026-06-30': 60000,
    '2026-11-15': 35000,
    '2026-12-31': 40000,
    '2027-06-30': 55000,
    '2027-12-31': 75000
}

btc_series = pd.Series(btc_puntos, index=pd.to_datetime(list(btc_puntos.keys())))
btc_series = btc_series.reindex(fechas).interpolate(method='cubic')

# Proyección NASDAQ
nasdaq_puntos = {
    '2024-01-01': 16000,
    '2024-12-31': 17500,
    '2025-06-30': 18500,
    '2025-12-31': 18000,
    '2026-06-30': 15500,
    '2026-12-31': 14500,
    '2027-06-30': 16500,
    '2027-12-31': 18000
}

nasdaq_series = pd.Series(nasdaq_puntos, index=pd.to_datetime(list(nasdaq_puntos.keys())))
nasdaq_series = nasdaq_series.reindex(fechas).interpolate(method='cubic')

df_timeline = pd.DataFrame({
    'Fecha': fechas,
    'Bitcoin_Proyeccion': btc_series.values,
    'NASDAQ_Proyeccion': nasdaq_series.values,
    'Año': fechas.year,
    'Trimestre': fechas.quarter,
    'Mes': fechas.month,
    'Año_Trimestre': fechas.to_period('Q').astype(str)
})

archivo = os.path.join(ruta_powerbi, 'timeline_proyeccion_2024_2027.csv')
df_timeline.to_csv(archivo, index=False, encoding='utf-8-sig')
print(f"   ✅ timeline_proyeccion_2024_2027.csv ({len(df_timeline)} filas)")

# ============================================================================
# TABLA 3: SEMÁFORO DE RIESGO TRIMESTRAL
# ============================================================================
print("📋 Tabla 3: Semáforo de Riesgo Trimestral")

df_semaforo = pd.DataFrame({
    'Periodo': ['2024 Q4', '2025 Q1', '2025 Q2', '2025 Q3', '2025 Q4',
                '2026 Q1', '2026 Q2', '2026 Q3', '2026 Q4',
                '2027 Q1', '2027 Q2', '2027 Q3', '2027 Q4'],
    'Año': [2024, 2025, 2025, 2025, 2025, 2026, 2026, 2026, 2026, 2027, 2027, 2027, 2027],
    'Trimestre': [4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
    'Exposicion_Crypto_Pct': [70, 70, 70, 40, 20, 10, 10, 10, 10, 10, 35, 50, 65],
    'Exposicion_NASDAQ_Pct': [30, 30, 30, 40, 30, 10, 10, 10, 10, 20, 35, 30, 25],
    'Exposicion_Cash_Pct': [0, 0, 0, 20, 50, 80, 80, 80, 80, 70, 30, 20, 10],
    'Nivel_Riesgo': ['Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto', 
                     'Crítico', 'Crítico', 'Crítico', 'Crítico',
                     'Alto', 'Medio', 'Bajo', 'Bajo'],
    'Color_Semaforo': ['Verde', 'Verde', 'Amarillo', 'Naranja', 'Rojo',
                       'Rojo Oscuro', 'Rojo Oscuro', 'Rojo Oscuro', 'Rojo Oscuro',
                       'Amarillo', 'Verde', 'Verde', 'Verde'],
    'Accion_Recomendada': ['MANTENER', 'MANTENER', 'VIGILAR', 'REDUCIR', 'VENDER',
                          'DEFENSIVA', 'DEFENSIVA', 'DEFENSIVA', 'DEFENSIVA',
                          'PREPARAR', 'COMPRAR', 'COMPRAR', 'MANTENER']
})

archivo = os.path.join(ruta_powerbi, 'semaforo_riesgo_trimestral.csv')
df_semaforo.to_csv(archivo, index=False, encoding='utf-8-sig')
print(f"   ✅ semaforo_riesgo_trimestral.csv ({len(df_semaforo)} filas)")

# ============================================================================
# TABLA 4: CICLO PRESIDENCIAL USA
# ============================================================================
print("📋 Tabla 4: Ciclo Presidencial USA")

df_presidencial = pd.DataFrame({
    'Año_Ciclo': [1, 2, 3, 4],
    'Año_Tipo': ['Post-Elección', 'Mid-term', 'Pre-Elección', 'Elección'],
    'SP500_Retorno_Promedio': [6.5, 4.2, 16.3, 7.5],
    'NASDAQ_Retorno_Promedio': [8.2, 2.1, 22.7, 11.3],
    'Probabilidad_Positivo': [63, 58, 88, 75],
    'Ejemplo_2021_2024': [2021, 2022, 2023, 2024],
    'NASDAQ_Real_2021_2024': [21.4, -33.1, 43.4, 24.9]
})

archivo = os.path.join(ruta_powerbi, 'ciclo_presidencial_usa.csv')
df_presidencial.to_csv(archivo, index=False, encoding='utf-8-sig')
print(f"   ✅ ciclo_presidencial_usa.csv ({len(df_presidencial)} filas)")

# ============================================================================
# TABLA 5: FASES DEL CICLO BITCOIN
# ============================================================================
print("📋 Tabla 5: Fases del Ciclo Bitcoin")

df_fases = pd.DataFrame({
    'Fase_Numero': [1, 2, 3, 4],
    'Fase_Nombre': ['Pre-Halving', 'Bull Market', 'Bear Market', 'Acumulación'],
    'Duracion_Meses': [12, 18, 12, 6],
    'Retorno_Tipico_Pct': [100, 400, -75, 10],
    'Volatilidad': ['Media', 'Alta', 'Muy Alta', 'Baja'],
    'Descripcion': [
        'Anticipación al halving, acumulación institucional',
        'Máxima euforia, ATH, FOMO retail',
        'Corrección profunda, capitulación',
        'Silencio mediático, oportunidad de compra'
    ],
    'Estrategia_Recomendada': [
        'Acumular gradualmente',
        'Tomar ganancias parciales',
        'Mantener cash, aguantar',
        'Comprar agresivamente'
    ]
})

archivo = os.path.join(ruta_powerbi, 'fases_ciclo_bitcoin.csv')
df_fases.to_csv(archivo, index=False, encoding='utf-8-sig')
print(f"   ✅ fases_ciclo_bitcoin.csv ({len(df_fases)} filas)")

# ============================================================================
# TABLA 6: EVENTOS CLAVE 2024-2027
# ============================================================================
print("📋 Tabla 6: Eventos Clave 2024-2027")

df_eventos = pd.DataFrame({
    'Fecha': [
        '2024-04-20', '2024-11-05', '2025-10-15', 
        '2026-01-20', '2026-11-15', '2027-07-01'
    ],
    'Evento': [
        'Halving Bitcoin #4',
        'Elecciones USA 2024',
        'ATH Bitcoin Proyectado',
        'Nueva Administración USA',
        'Fondo Bitcoin Proyectado',
        'Inicio Recuperación'
    ],
    'Tipo_Evento': [
        'Bitcoin', 'Político', 'Bitcoin', 
        'Político', 'Bitcoin', 'Bitcoin'
    ],
    'Impacto_Esperado': [
        'Muy Positivo', 'Neutral', 'Máximo Positivo',
        'Negativo', 'Máximo Negativo', 'Positivo'
    ],
    'Bitcoin_Precio_Estimado': [64000, 72000, 160000, 110000, 35000, 55000],
    'Accion_Sugerida': [
        'Mantener',
        'Vigilar',
        'VENDER 60-70%',
        'Mantener cash',
        'COMPRAR agresivo',
        'Acumular'
    ]
})

df_eventos['Fecha'] = pd.to_datetime(df_eventos['Fecha'])
archivo = os.path.join(ruta_powerbi, 'eventos_clave_2024_2027.csv')
df_eventos.to_csv(archivo, index=False, encoding='utf-8-sig')
print(f"   ✅ eventos_clave_2024_2027.csv ({len(df_eventos)} filas)")

# ============================================================================
# RESUMEN
# ============================================================================
print("\n" + "="*80)
print("✅ TABLAS CSV GENERADAS PARA POWER BI")
print("="*80)
print(f"\n📁 Ubicación: {os.path.abspath(ruta_powerbi)}")
print("\n📊 Archivos creados:")
print("   1. ciclos_bitcoin_halvings.csv")
print("   2. timeline_proyeccion_2024_2027.csv")
print("   3. semaforo_riesgo_trimestral.csv")
print("   4. ciclo_presidencial_usa.csv")
print("   5. fases_ciclo_bitcoin.csv")
print("   6. eventos_clave_2024_2027.csv")
print("\n💡 Listas para importar a Power BI")
print("="*80 + "\n")


📊 GENERANDO TABLAS CSV PARA POWER BI - CICLOS TEMPORALES
📁 Carpeta destino: c:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final\data\powerbi

📋 Tabla 1: Ciclos Bitcoin - Halvings
   ✅ ciclos_bitcoin_halvings.csv (4 filas)
📋 Tabla 2: Timeline Proyección 2024-2027
   ✅ timeline_proyeccion_2024_2027.csv (1461 filas)
📋 Tabla 3: Semáforo de Riesgo Trimestral
   ✅ semaforo_riesgo_trimestral.csv (13 filas)
📋 Tabla 4: Ciclo Presidencial USA
   ✅ ciclo_presidencial_usa.csv (4 filas)
📋 Tabla 5: Fases del Ciclo Bitcoin
   ✅ fases_ciclo_bitcoin.csv (4 filas)
📋 Tabla 6: Eventos Clave 2024-2027
   ✅ eventos_clave_2024_2027.csv (6 filas)

✅ TABLAS CSV GENERADAS PARA POWER BI

📁 Ubicación: c:\Users\USER\Desktop\Thepower dataanalytic\Proyectos The power\proyecto-final\data\powerbi

📊 Archivos creados:
   1. ciclos_bitcoin_halvings.csv
   2. timeline_proyeccion_2024_2027.csv
   3. semaforo_riesgo_trimestral.csv
   4. ciclo_presidencial_usa.csv
   5. fases_ciclo_bitcoin.csv
  